In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
import csv
import json
print('All required libraries have been successfully imported.')


All required libraries have been successfully imported.


In [3]:

with open('Testing/Project-Dataset/jaipur_attraction_urls.txt', newline='') as f:
    reader = csv.reader(f)
    attraction_full_urllist = list(reader)

print('Attraction urls file read successfully..')
print(f'Number of Attractions: {len(attraction_full_urllist)}')

Attraction urls file read successfully..
Number of Attractions: 1722


In [18]:
attraction_info = []
for iter_val in range(len(attraction_full_urllist)):
    url = str(attraction_full_urllist[iter_val])[2:-2]

    # Get the attraction page from url and create a beautiful soup instance
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'lxml')

    # BASIC INFORMATION
    div1 = soup.find('div', class_ = 'uitk-flex uitk-flex-column uitk-layout-grid-item uitk-layout-grid-item-columnspan-small-12 uitk-layout-grid-item-columnspan-large-4 uitk-card all-r-padding-six')

    attraction_name = div1.find('h1').text
    try:
        attraction_rating = div1.find('div', class_ = 'uitk-text uitk-type-500 uitk-type-bold uitk-flex-align-self-center uitk-flex-item uitk-text-default-theme').text[0:-2]
    except:
        attraction_rating = ''
    attraction_price = div1.find('span', class_ = 'uitk-lockup-price').text

    # FEATURES
    features_div = soup.find('div', class_ = 'uitk-layout-grid uitk-layout-grid-gap-one uitk-layout-grid-columns-12')
    features_div = features_div.find_all('div', class_ = 'uitk-text uitk-type-300 uitk-spacing uitk-spacing-margin-inlinestart-one uitk-text-default-theme')

    attraction_features = []

    for feature in features_div:
        attraction_features.append(feature.text)
    
    # LOCATION
    location = soup.find('img', class_ = "uitk-image-media uitk-card-roundcorner-all padding-bottom-zero")

    location_link = location.attrs['src']
    coords = re.findall('%7C[0-9]+.[0-9]+,[0-9]+.[0-9]+', location_link).pop(1)
    coords = coords[3:]
    latitude, longitude = coords.split(",")

    # CATEGORY
    attraction_category = re.findall('.categories=\w+', url)[0].replace('.categories=', '')

    attraction_info.append([iter_val+1, attraction_name, attraction_category, attraction_rating, attraction_price, latitude, longitude, "Jaipur", "India"])
    if iter_val%50 == 0:
        print(iter_val)

print(f"Extracted info of {len(attraction_info)} attractions")


0
50
100
150
200
250
300
350
400
450
500
550
600


AttributeError: 'NoneType' object has no attribute 'find'

In [19]:
def create_csv(fields, rows, dirname, filename):
    with open(dirname+filename, 'w', encoding='UTF8', newline='') as csvfile: 
        # creating a csv writer object
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(fields) 
        csvwriter.writerows(rows)
    print(f"Saved csv with name {filename}")

In [20]:
fields = ['attraction_id', 'attraction_name','attraction_category', 'attraction_rating', 'attraction_price', 'latitude', 'longitude', 'city', 'country']
create_csv(fields, attraction_info, "Testing/Project-Dataset/", "attraction_info.csv")
print('Hotel Dataframe updated..')

Saved csv with name attraction_info.csv
Hotel Dataframe updated..


In [21]:
def make_json(csvFilePath, jsonFilePath):
    
    # create a dictionary
    data = ""

    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)

        # Convert each row into a dictionary
        # and add it to data
        for rows in csvReader:

            # Assuming a column named 'No' to
            # be the primary key
            
            data = data+'\n'
 
    # Open a json writer, and use the json.dumps()
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data ,indent=1))

# Driver Code
 
# Decide the two file paths according to your
# computer system
csvFilePath = r'Testing/Project-Dataset/hotel_info.csv'
jsonFilePath = r'Testing/Project-Dataset/hotel_info3.json'
 
# Call the make_json function
make_json(csvFilePath, jsonFilePath)